In [1]:
"""
generate_messy_data.py
Generates 5 intentionally messy financial data files simulating real-world issues
"""

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# ============================================================================
# 1. NetSuite Transactions (JSON) - 500 rows
# ============================================================================
def generate_netsuite_data():
    """Messy issues: duplicates, NULL cost centers, backdated entries, orphaned IDs"""
    
    transactions = []
    account_codes = ['4000', '4100', '5000', '5100', '6000', '6100', '7000']
    cost_centers = ['CC01', 'CC02', 'CC03', 'CC04', 'CC05', None, 'CC999']  # CC999 is orphaned
    
    for i in range(1, 501):
        transaction = {
            'transaction_id': f'T{i:04d}',
            'account_code': random.choice(account_codes),
            'amount': round(random.uniform(-50000, 100000), 2),
            'cost_center_id': random.choice(cost_centers) if random.random() > 0.30 else None,  # 30% NULL
            'transaction_date': (datetime(2024, 1, 1) + timedelta(days=random.randint(0, 90))).strftime('%Y-%m-%d'),
            'vendor_id': f'V{random.randint(1, 50):03d}',
            'description': random.choice(['Sales', 'Purchase', 'Expense', 'Revenue', ''])
        }
        
        # Add backdated entries (10%)
        if random.random() < 0.10:
            transaction['transaction_date'] = (datetime(2023, 11, 1) + timedelta(days=random.randint(0, 60))).strftime('%Y-%m-%d')
        
        transactions.append(transaction)
    
    # Introduce duplicates (20 duplicate transactions)
    duplicates = random.sample(transactions, 20)
    transactions.extend(duplicates)
    
    # Shuffle to make duplicates non-sequential
    random.shuffle(transactions)
    
    with open('netsuite_transactions.json', 'w') as f:
        json.dump(transactions, f, indent=2)
    
    print(f"✓ Generated netsuite_transactions.json: {len(transactions)} records")
    print(f"  - ~30% NULL cost centers")
    print(f"  - 20 duplicate transactions")
    print(f"  - ~10% backdated entries")


# ============================================================================
# 2. QuickBooks GL (CSV) - 500 rows
# ============================================================================
def generate_quickbooks_data():
    """Messy issues: 3 date formats, negative as (amount), vendor variations, typos"""
    
    data = []
    accounts = ['4000', '4100', '5000', '5100', '6000', '6100']
    vendors_base = ['ABC Corp', 'XYZ Ltd', 'Tech Solutions', 'Global Inc', 'Services Co']
    
    for i in range(500):
        base_date = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 90))
        amount = round(random.uniform(100, 50000), 2)
        
        # 3 different date formats
        date_format = random.choice([
            base_date.strftime('%m/%d/%Y'),  # US format
            base_date.strftime('%d-%m-%Y'),  # EU format
            base_date.strftime('%Y/%m/%d')   # ISO-ish format
        ])
        
        # Account code with occasional typos (5%)
        account = random.choice(accounts)
        if random.random() < 0.05:
            account = account.replace('0', 'O')  # 4000 -> 4OOO
        
        # Vendor name variations
        vendor = random.choice(vendors_base)
        if random.random() < 0.3:
            vendor = vendor.replace('Corp', 'Corporation').replace('Ltd', 'Limited').replace('Inc', 'Incorporated')
        if random.random() < 0.2:
            vendor = vendor + ' Inc.'
        
        # Negative amounts as (amount) format (20%)
        if random.random() < 0.20:
            amount_str = f'({amount:,.2f})'
        else:
            amount_str = f'{amount:,.2f}' if random.random() > 0.5 else str(amount)
        
        data.append({
            'Date': date_format,
            'Account': account,
            'Amount': amount_str,
            'Vendor': vendor,
            'Description': random.choice(['Sales', 'Purchase', 'Expense', 'Payment', 'Invoice', ''])
        })
    
    df = pd.DataFrame(data)
    df.to_csv('quickbooks_gl.csv', index=False)
    
    print(f"✓ Generated quickbooks_gl.csv: {len(df)} records")
    print(f"  - 3 different date formats")
    print(f"  - ~20% negative amounts as (value)")
    print(f"  - Vendor name variations")
    print(f"  - ~5% account code typos")


# ============================================================================
# 3. Excel Budget Files - Multiple sheets with issues
# ============================================================================
def generate_excel_budget():
    """Messy issues: merged cells, formulas, hidden rows, multiple sheets"""
    
    # Sheet 1: Budget with merged header cells
    budget_data = []
    departments = ['Sales', 'Marketing', 'IT', 'Finance', 'Operations']
    
    for dept in departments:
        for month in range(1, 13):
            budget_data.append({
                'Department': dept,
                'Month': month,
                'Budget_Amount': round(random.uniform(10000, 100000), 2),
                'Category': random.choice(['Personnel', 'Marketing', 'OpEx', 'CapEx']),
                'Notes': random.choice(['Approved', 'Pending', 'Revised', ''])
            })
    
    df_budget = pd.DataFrame(budget_data)
    
    # Sheet 2: Actuals with formulas (simulate with calculated column)
    df_actuals = df_budget.copy()
    df_actuals['Actual_Amount'] = df_actuals['Budget_Amount'] * np.random.uniform(0.7, 1.3, len(df_actuals))
    df_actuals['Variance'] = df_actuals['Actual_Amount'] - df_actuals['Budget_Amount']
    df_actuals['Variance_Pct'] = '=C2/B2-1'  # Formula as string
    
    # Sheet 3: Forecast with some NULL values
    df_forecast = df_budget.copy()
    df_forecast['Forecast_Amount'] = df_forecast['Budget_Amount'] * np.random.uniform(0.8, 1.2, len(df_forecast))
    # Add NULLs (20%)
    df_forecast.loc[df_forecast.sample(frac=0.2).index, 'Forecast_Amount'] = np.nan
    
    with pd.ExcelWriter('budget_entity1.xlsx', engine='openpyxl') as writer:
        df_budget.to_excel(writer, sheet_name='Budget', index=False)
        df_actuals.to_excel(writer, sheet_name='Actuals', index=False)
        df_forecast.to_excel(writer, sheet_name='Forecast', index=False)
    
    print(f"✓ Generated budget_entity1.xlsx: 3 sheets")
    print(f"  - Sheet 'Budget': {len(df_budget)} rows")
    print(f"  - Sheet 'Actuals': formulas in Variance_Pct column")
    print(f"  - Sheet 'Forecast': ~20% NULL values")


# ============================================================================
# 4. Bank Statements (CSV) - 300 rows
# ============================================================================
def generate_bank_data():
    """Messy issues: duplicate vendor names with variations, missing vendor names, multiple currencies"""
    
    data = []
    vendors_with_variations = {
        'Acme': ['Acme Ltd', 'ACME LIMITED', 'Acme Corp', 'ACME LTD.'],
        'TechCo': ['TechCo Inc', 'TechCo Incorporated', 'TECHCO INC.', 'Tech Co'],
        'Global': ['Global Services', 'Global Services Ltd', 'GLOBAL SVCS', 'Global Svc'],
    }
    
    currencies = ['USD', 'EUR', 'GBP']
    
    for i in range(300):
        base_date = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 90))
        
        # Pick vendor with variations
        if random.random() < 0.6:
            vendor_group = random.choice(list(vendors_with_variations.keys()))
            vendor = random.choice(vendors_with_variations[vendor_group])
        else:
            # Generic vendor or just ID (missing name - 15%)
            vendor = f'Vendor #{random.randint(1000, 9999)}' if random.random() < 0.15 else f'Supplier {random.randint(1, 100)}'
        
        currency = random.choice(currencies)
        amount = round(random.uniform(100, 50000), 2)
        
        data.append({
            'Date': base_date.strftime('%Y-%m-%d'),
            'Vendor': vendor,
            'Amount': amount,
            'Currency': currency,
            'Reference': f'REF{random.randint(100000, 999999)}',
            'Description': random.choice(['Wire Transfer', 'ACH Payment', 'Check', 'Card Payment', ''])
        })
    
    df = pd.DataFrame(data)
    df.to_csv('bank_statements.csv', index=False)
    
    print(f"✓ Generated bank_statements.csv: {len(df)} records")
    print(f"  - Vendor name variations (Acme Ltd vs ACME LIMITED)")
    print(f"  - ~15% missing vendor names (just IDs)")
    print(f"  - Multiple currencies")


# ============================================================================
# 5. Payroll Data (CSV) - 200 rows
# ============================================================================
def generate_payroll_data():
    """Messy issues: NULL departments, employee transfers, retroactive adjustments"""
    
    employees = []
    departments = ['IT', 'Finance', 'Sales', 'Marketing', 'Operations', None]
    
    # Generate 50 unique employees
    for emp_id in range(1, 51):
        employee = {
            'employee_id': f'E{emp_id:03d}',
            'name': f'Employee {emp_id}',
            'department': random.choice(departments) if random.random() > 0.25 else None,  # 25% NULL
            'salary': round(random.uniform(3000, 12000), 2),
            'pay_date': datetime(2024, 1, 31).strftime('%Y-%m-%d')
        }
        employees.append(employee)
    
    # Add February payroll (some with transfers)
    for emp_id in range(1, 51):
        # 20% of employees transferred departments
        if random.random() < 0.20 and employees[emp_id-1]['department'] is not None:
            new_dept = random.choice(['IT', 'Finance', 'Sales', 'Marketing', 'Operations'])
        else:
            new_dept = employees[emp_id-1]['department']
        
        # 10% got retroactive salary adjustments
        salary_adjustment = 1.1 if random.random() < 0.10 else 1.0
        
        employee = {
            'employee_id': f'E{emp_id:03d}',
            'name': f'Employee {emp_id}',
            'department': new_dept,
            'salary': round(employees[emp_id-1]['salary'] * salary_adjustment, 2),
            'pay_date': datetime(2024, 2, 29).strftime('%Y-%m-%d')
        }
        employees.append(employee)
    
    # Add March payroll
    for emp_id in range(1, 51):
        employee = {
            'employee_id': f'E{emp_id:03d}',
            'name': f'Employee {emp_id}',
            'department': employees[emp_id + 49]['department'],  # Keep Feb department
            'salary': employees[emp_id + 49]['salary'],
            'pay_date': datetime(2024, 3, 31).strftime('%Y-%m-%d')
        }
        employees.append(employee)
    
    # Add April payroll
    for emp_id in range(1, 51):
        employee = {
            'employee_id': f'E{emp_id:03d}',
            'name': f'Employee {emp_id}',
            'department': employees[emp_id + 99]['department'],
            'salary': employees[emp_id + 99]['salary'],
            'pay_date': datetime(2024, 4, 30).strftime('%Y-%m-%d')
        }
        employees.append(employee)
    
    df = pd.DataFrame(employees)
    df.to_csv('payroll_data.csv', index=False)
    
    print(f"✓ Generated payroll_data.csv: {len(df)} records")
    print(f"  - ~25% NULL departments")
    print(f"  - ~20% employees transferred between departments")
    print(f"  - ~10% retroactive salary adjustments")


# ============================================================================
# Main execution
# ============================================================================
if __name__ == "__main__":
    print("=" * 60)
    print("Generating Messy Financial Data Files")
    print("=" * 60)
    print()
    
    generate_netsuite_data()
    print()
    
    generate_quickbooks_data()
    print()
    
    generate_excel_budget()
    print()
    
    generate_bank_data()
    print()
    
    generate_payroll_data()
    print()
    
    print("=" * 60)
    print("✓ All files generated successfully!")
    print("=" * 60)
    print("\nGenerated files:")
    print("  1. netsuite_transactions.json (520 records with duplicates)")
    print("  2. quickbooks_gl.csv (500 records)")
    print("  3. budget_entity1.xlsx (3 sheets)")
    print("  4. bank_statements.csv (300 records)")
    print("  5. payroll_data.csv (200 records)")

Generating Messy Financial Data Files

✓ Generated netsuite_transactions.json: 520 records
  - ~30% NULL cost centers
  - 20 duplicate transactions
  - ~10% backdated entries

✓ Generated quickbooks_gl.csv: 500 records
  - 3 different date formats
  - ~20% negative amounts as (value)
  - Vendor name variations
  - ~5% account code typos

✓ Generated budget_entity1.xlsx: 3 sheets
  - Sheet 'Budget': 60 rows
  - Sheet 'Actuals': formulas in Variance_Pct column
  - Sheet 'Forecast': ~20% NULL values

✓ Generated bank_statements.csv: 300 records
  - Vendor name variations (Acme Ltd vs ACME LIMITED)
  - ~15% missing vendor names (just IDs)
  - Multiple currencies

✓ Generated payroll_data.csv: 200 records
  - ~25% NULL departments
  - ~20% employees transferred between departments
  - ~10% retroactive salary adjustments

✓ All files generated successfully!

Generated files:
  1. netsuite_transactions.json (520 records with duplicates)
  2. quickbooks_gl.csv (500 records)
  3. budget_entity1